<a href="https://colab.research.google.com/github/SanaAlotaibi/DataScienceProject/blob/main/Data_Collection_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data collection code
We used the same code for every year separately. they were executed using diffrent tokens by diffrent students.

The actual tokens were written instead of the word "Token" in the 7th line but due to security reasons they got replaced in the code.


* 2020


In [ ]:
import requests
import pandas as pd
import time
import random
import re

# ==== CONFIG ====
GITHUB_TOKEN = 'Token'  # حطو حقكم
HEADERS = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
    "User-Agent": "Data-Science-Project-KSUM"
}

QUERY = 'AI OR artificial-intelligence OR machine-learning OR generative-ai'
PER_PAGE = 100
REPOS_PER_QUARTER = 250  # target per quarter
QUARTERS = [
    ('2020-01-01', '2020-03-31'),
    ('2020-04-01', '2020-06-30'),
    ('2020-07-01', '2020-09-30'),
    ('2020-10-01', '2020-12-31')
]#غيرو السنه للسنه حقتكم

# ==== FUNCTIONS ====

def get_repo_details(full_name):
    """Fetch detailed repo info: languages, topics, license, contributors"""
    repo_details = {'languages': {}, 'topics': [], 'license': None, 'contributors_count': 0}

    # Languages
    languages_url = f"https://api.github.com/repos/{full_name}/languages"
    languages_res = requests.get(languages_url, headers=HEADERS)
    if languages_res.status_code == 200:
        repo_details['languages'] = languages_res.json()

    # Topics
    topics_url = f"https://api.github.com/repos/{full_name}/topics"
    topics_headers = HEADERS.copy()
    topics_headers['Accept'] = 'application/vnd.github.mercy-preview+json'
    topics_res = requests.get(topics_url, headers=topics_headers)
    if topics_res.status_code == 200:
        repo_details['topics'] = topics_res.json().get('names', [])

    # License
    repo_url = f"https://api.github.com/repos/{full_name}"
    repo_res = requests.get(repo_url, headers=HEADERS)
    if repo_res.status_code == 200:
        repo_data = repo_res.json()
        if 'license' in repo_data and repo_data['license']:
            repo_details['license'] = repo_data['license'].get('spdx_id')

    # Contributors count
    contributors_url = f"https://api.github.com/repos/{full_name}/contributors?per_page=1"
    contributors_res = requests.get(contributors_url, headers=HEADERS)
    if contributors_res.status_code == 200:
        if 'Link' in contributors_res.headers and 'last' in contributors_res.headers['Link']:
            last_page_url = re.search(r'<(.+)>; rel="last"', contributors_res.headers['Link']).group(1)
            last_page_number = int(re.search(r'page=(\d+)', last_page_url).group(1))
            repo_details['contributors_count'] = last_page_number
        else:
            repo_details['contributors_count'] = len(contributors_res.json())

    return repo_details

def fetch_top_repositories(query, start_date, end_date, per_page=30, max_repos=250):
    """Fetch repositories sorted by stars in a given date range"""
    repos = []
    page = 1
    while len(repos) < max_repos:
        search_query = f'{query} created:{start_date}..{end_date}'
        url = "https://api.github.com/search/repositories"
        params = {
            'q': search_query,
            'sort': 'stars',
            'order': 'desc',
            'per_page': per_page,
            'page': page
        }
        response = requests.get(url, headers=HEADERS, params=params)
        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            break

        items = response.json().get('items', [])
        if not items:
            break

        repos.extend(items)
        page += 1
        time.sleep(1)

    return repos[:max_repos]

# ==== MAIN EXECUTION ====
if __name__ == '__main__':
    all_final_data = []

    for start_date, end_date in QUARTERS:
        print(f"🔍 Fetching top {REPOS_PER_QUARTER} repos from {start_date} to {end_date}...")
        quarter_repos = fetch_top_repositories(QUERY, start_date, end_date, PER_PAGE, REPOS_PER_QUARTER)

        print(f"📦 Collecting detailed info for {len(quarter_repos)} repos...")
        for item in quarter_repos:
            owner = item.get('owner', {}).get('login')
            name = item.get('name')
            full_name = f"{owner}/{name}"

            repo_details = get_repo_details(full_name)

            repo_info = {
                'name': name,
                'owner': owner,
                'url': item.get('html_url'),
                'description': item.get('description'),
                'stars': item.get('stargazers_count'),
                'forks': item.get('forks_count'),
                'created_at': item.get('created_at'),
                'pushed_at': item.get('pushed_at'),
                'primary_language': item.get('language'),
                'all_languages_bytes': repo_details['languages'],
                'topics': ", ".join(repo_details['topics']),
                'contributors_count': repo_details['contributors_count']
            }

            all_final_data.append(repo_info)
            time.sleep(0.5)

    df = pd.DataFrame(all_final_data)
    output_filename = "github_ai_repos_2020_quarters.csv"#سموه باسم السنه حقتكم
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"\n✅ Completed! Saved {len(df)} repositories to {output_filename}")


🔍 Fetching top 251 repos from 2020-01-01 to 2020-03-31...
📦 Collecting detailed info for 251 repos...
🔍 Fetching top 251 repos from 2020-04-01 to 2020-06-30...
📦 Collecting detailed info for 251 repos...
🔍 Fetching top 251 repos from 2020-07-01 to 2020-09-30...
📦 Collecting detailed info for 251 repos...
🔍 Fetching top 251 repos from 2020-10-01 to 2020-12-31...
📦 Collecting detailed info for 251 repos...

✅ Completed! Saved 1004 repositories to github_ai_repos_2020_quarters.csv


* 2021

In [ ]:
import requests
import pandas as pd
import time
import random
import re

# ==== CONFIG ====
GITHUB_TOKEN = 'Token'  # حطو حقكم
HEADERS = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
    "User-Agent": "Data-Science-Project-KSUM"
}

QUERY = 'AI OR artificial-intelligence OR machine-learning OR generative-ai'
PER_PAGE = 100
REPOS_PER_QUARTER = 250  # target per quarter
QUARTERS = [
    ('2021-01-01', '2021-03-31'),
    ('2021-04-01', '2021-06-30'),
    ('2021-07-01', '2021-09-30'),
    ('2021-10-01', '2021-12-31')
]#غيرو السنه للسنه حقتكم

# ==== FUNCTIONS ====

def get_repo_details(full_name):
    """Fetch detailed repo info: languages, topics, license, contributors"""
    repo_details = {'languages': {}, 'topics': [], 'license': None, 'contributors_count': 0}

    # Languages
    languages_url = f"https://api.github.com/repos/{full_name}/languages"
    languages_res = requests.get(languages_url, headers=HEADERS)
    if languages_res.status_code == 200:
        repo_details['languages'] = languages_res.json()

    # Topics
    topics_url = f"https://api.github.com/repos/{full_name}/topics"
    topics_headers = HEADERS.copy()
    topics_headers['Accept'] = 'application/vnd.github.mercy-preview+json'
    topics_res = requests.get(topics_url, headers=topics_headers)
    if topics_res.status_code == 200:
        repo_details['topics'] = topics_res.json().get('names', [])

    # License
    repo_url = f"https://api.github.com/repos/{full_name}"
    repo_res = requests.get(repo_url, headers=HEADERS)
    if repo_res.status_code == 200:
        repo_data = repo_res.json()
        if 'license' in repo_data and repo_data['license']:
            repo_details['license'] = repo_data['license'].get('spdx_id')

    # Contributors count
    contributors_url = f"https://api.github.com/repos/{full_name}/contributors?per_page=1"
    contributors_res = requests.get(contributors_url, headers=HEADERS)
    if contributors_res.status_code == 200:
        if 'Link' in contributors_res.headers and 'last' in contributors_res.headers['Link']:
            last_page_url = re.search(r'<(.+)>; rel="last"', contributors_res.headers['Link']).group(1)
            last_page_number = int(re.search(r'page=(\d+)', last_page_url).group(1))
            repo_details['contributors_count'] = last_page_number
        else:
            repo_details['contributors_count'] = len(contributors_res.json())

    return repo_details

def fetch_top_repositories(query, start_date, end_date, per_page=30, max_repos=250):
    """Fetch repositories sorted by stars in a given date range"""
    repos = []
    page = 1
    while len(repos) < max_repos:
        search_query = f'{query} created:{start_date}..{end_date}'
        url = "https://api.github.com/search/repositories"
        params = {
            'q': search_query,
            'sort': 'stars',
            'order': 'desc',
            'per_page': per_page,
            'page': page
        }
        response = requests.get(url, headers=HEADERS, params=params)
        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            break

        items = response.json().get('items', [])
        if not items:
            break

        repos.extend(items)
        page += 1
        time.sleep(1)

    return repos[:max_repos]

# ==== MAIN EXECUTION ====
if __name__ == '__main__':
    all_final_data = []

    for start_date, end_date in QUARTERS:
        print(f"🔍 Fetching top {REPOS_PER_QUARTER} repos from {start_date} to {end_date}...")
        quarter_repos = fetch_top_repositories(QUERY, start_date, end_date, PER_PAGE, REPOS_PER_QUARTER)

        print(f"📦 Collecting detailed info for {len(quarter_repos)} repos...")
        for item in quarter_repos:
            owner = item.get('owner', {}).get('login')
            name = item.get('name')
            full_name = f"{owner}/{name}"

            repo_details = get_repo_details(full_name)

            repo_info = {
                'name': name,
                'owner': owner,
                'url': item.get('html_url'),
                'description': item.get('description'),
                'stars': item.get('stargazers_count'),
                'forks': item.get('forks_count'),
                'created_at': item.get('created_at'),
                'pushed_at': item.get('pushed_at'),
                'primary_language': item.get('language'),
                'all_languages_bytes': repo_details['languages'],
                'topics': ", ".join(repo_details['topics']),
                'contributors_count': repo_details['contributors_count']
            }

            all_final_data.append(repo_info)
            time.sleep(0.5)

    df = pd.DataFrame(all_final_data)
    output_filename = "github_ai_repos_2021_quarters.csv"#سموه باسم السنه حقتكم
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"\n✅ Completed! Saved {len(df)} repositories to {output_filename}")

* 2022

In [ ]:
import requests
import pandas as pd
import time
import random
import re

# ==== CONFIG ====
GITHUB_TOKEN = 'Token'  # حطو حقكم
HEADERS = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
    "User-Agent": "Data-Science-Project-KSUM"
}

QUERY = 'AI OR artificial-intelligence OR machine-learning OR generative-ai'
PER_PAGE = 100
REPOS_PER_QUARTER = 250  # target per quarter
QUARTERS = [
    ('2022-01-01', '2022-03-31'),
    ('2022-04-01', '2022-06-30'),
    ('2022-07-01', '2022-09-30'),
    ('2022-10-01', '2022-12-31')
]#غيرو السنه للسنه حقتكم

# ==== FUNCTIONS ====

def get_repo_details(full_name):
    """Fetch detailed repo info: languages, topics, license, contributors"""
    repo_details = {'languages': {}, 'topics': [], 'license': None, 'contributors_count': 0}

    # Languages
    languages_url = f"https://api.github.com/repos/{full_name}/languages"
    languages_res = requests.get(languages_url, headers=HEADERS)
    if languages_res.status_code == 200:
        repo_details['languages'] = languages_res.json()

    # Topics
    topics_url = f"https://api.github.com/repos/{full_name}/topics"
    topics_headers = HEADERS.copy()
    topics_headers['Accept'] = 'application/vnd.github.mercy-preview+json'
    topics_res = requests.get(topics_url, headers=topics_headers)
    if topics_res.status_code == 200:
        repo_details['topics'] = topics_res.json().get('names', [])

    # License
    repo_url = f"https://api.github.com/repos/{full_name}"
    repo_res = requests.get(repo_url, headers=HEADERS)
    if repo_res.status_code == 200:
        repo_data = repo_res.json()
        if 'license' in repo_data and repo_data['license']:
            repo_details['license'] = repo_data['license'].get('spdx_id')

    # Contributors count
    contributors_url = f"https://api.github.com/repos/{full_name}/contributors?per_page=1"
    contributors_res = requests.get(contributors_url, headers=HEADERS)
    if contributors_res.status_code == 200:
        if 'Link' in contributors_res.headers and 'last' in contributors_res.headers['Link']:
            last_page_url = re.search(r'<(.+)>; rel="last"', contributors_res.headers['Link']).group(1)
            last_page_number = int(re.search(r'page=(\d+)', last_page_url).group(1))
            repo_details['contributors_count'] = last_page_number
        else:
            repo_details['contributors_count'] = len(contributors_res.json())

    return repo_details

def fetch_top_repositories(query, start_date, end_date, per_page=30, max_repos=250):
    """Fetch repositories sorted by stars in a given date range"""
    repos = []
    page = 1
    while len(repos) < max_repos:
        search_query = f'{query} created:{start_date}..{end_date}'
        url = "https://api.github.com/search/repositories"
        params = {
            'q': search_query,
            'sort': 'stars',
            'order': 'desc',
            'per_page': per_page,
            'page': page
        }
        response = requests.get(url, headers=HEADERS, params=params)
        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            break

        items = response.json().get('items', [])
        if not items:
            break

        repos.extend(items)
        page += 1
        time.sleep(1)

    return repos[:max_repos]

# ==== MAIN EXECUTION ====
if __name__ == '__main__':
    all_final_data = []

    for start_date, end_date in QUARTERS:
        print(f"🔍 Fetching top {REPOS_PER_QUARTER} repos from {start_date} to {end_date}...")
        quarter_repos = fetch_top_repositories(QUERY, start_date, end_date, PER_PAGE, REPOS_PER_QUARTER)

        print(f"📦 Collecting detailed info for {len(quarter_repos)} repos...")
        for item in quarter_repos:
            owner = item.get('owner', {}).get('login')
            name = item.get('name')
            full_name = f"{owner}/{name}"

            repo_details = get_repo_details(full_name)

            repo_info = {
                'name': name,
                'owner': owner,
                'url': item.get('html_url'),
                'description': item.get('description'),
                'stars': item.get('stargazers_count'),
                'forks': item.get('forks_count'),
                'created_at': item.get('created_at'),
                'pushed_at': item.get('pushed_at'),
                'primary_language': item.get('language'),
                'all_languages_bytes': repo_details['languages'],
                'topics': ", ".join(repo_details['topics']),
                'contributors_count': repo_details['contributors_count']
            }

            all_final_data.append(repo_info)
            time.sleep(0.5)

    df = pd.DataFrame(all_final_data)
    output_filename = "github_ai_repos_2022_quarters.csv"#سموه باسم السنه حقتكم
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"\n✅ Completed! Saved {len(df)} repositories to {output_filename}")

* 2023

In [ ]:
import requests
import pandas as pd
import time
import random
import re

# ==== CONFIG ====
GITHUB_TOKEN = 'Token'  # حطو حقكم
HEADERS = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
    "User-Agent": "Data-Science-Project-KSUM"
}

QUERY = 'AI OR artificial-intelligence OR machine-learning OR generative-ai'
PER_PAGE = 100
REPOS_PER_QUARTER = 250  # target per quarter
QUARTERS = [
    ('2023-01-01', '2023-03-31'),
    ('2023-04-01', '2023-06-30'),
    ('2023-07-01', '2023-09-30'),
    ('2023-10-01', '2023-12-31')
]#غيرو السنه للسنه حقتكم

# ==== FUNCTIONS ====

def get_repo_details(full_name):
    """Fetch detailed repo info: languages, topics, license, contributors"""
    repo_details = {'languages': {}, 'topics': [], 'license': None, 'contributors_count': 0}

    # Languages
    languages_url = f"https://api.github.com/repos/{full_name}/languages"
    languages_res = requests.get(languages_url, headers=HEADERS)
    if languages_res.status_code == 200:
        repo_details['languages'] = languages_res.json()

    # Topics
    topics_url = f"https://api.github.com/repos/{full_name}/topics"
    topics_headers = HEADERS.copy()
    topics_headers['Accept'] = 'application/vnd.github.mercy-preview+json'
    topics_res = requests.get(topics_url, headers=topics_headers)
    if topics_res.status_code == 200:
        repo_details['topics'] = topics_res.json().get('names', [])

    # License
    repo_url = f"https://api.github.com/repos/{full_name}"
    repo_res = requests.get(repo_url, headers=HEADERS)
    if repo_res.status_code == 200:
        repo_data = repo_res.json()
        if 'license' in repo_data and repo_data['license']:
            repo_details['license'] = repo_data['license'].get('spdx_id')

    # Contributors count
    contributors_url = f"https://api.github.com/repos/{full_name}/contributors?per_page=1"
    contributors_res = requests.get(contributors_url, headers=HEADERS)
    if contributors_res.status_code == 200:
        if 'Link' in contributors_res.headers and 'last' in contributors_res.headers['Link']:
            last_page_url = re.search(r'<(.+)>; rel="last"', contributors_res.headers['Link']).group(1)
            last_page_number = int(re.search(r'page=(\d+)', last_page_url).group(1))
            repo_details['contributors_count'] = last_page_number
        else:
            repo_details['contributors_count'] = len(contributors_res.json())

    return repo_details

def fetch_top_repositories(query, start_date, end_date, per_page=30, max_repos=250):
    """Fetch repositories sorted by stars in a given date range"""
    repos = []
    page = 1
    while len(repos) < max_repos:
        search_query = f'{query} created:{start_date}..{end_date}'
        url = "https://api.github.com/search/repositories"
        params = {
            'q': search_query,
            'sort': 'stars',
            'order': 'desc',
            'per_page': per_page,
            'page': page
        }
        response = requests.get(url, headers=HEADERS, params=params)
        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            break

        items = response.json().get('items', [])
        if not items:
            break

        repos.extend(items)
        page += 1
        time.sleep(1)

    return repos[:max_repos]

# ==== MAIN EXECUTION ====
if __name__ == '__main__':
    all_final_data = []

    for start_date, end_date in QUARTERS:
        print(f"🔍 Fetching top {REPOS_PER_QUARTER} repos from {start_date} to {end_date}...")
        quarter_repos = fetch_top_repositories(QUERY, start_date, end_date, PER_PAGE, REPOS_PER_QUARTER)

        print(f"📦 Collecting detailed info for {len(quarter_repos)} repos...")
        for item in quarter_repos:
            owner = item.get('owner', {}).get('login')
            name = item.get('name')
            full_name = f"{owner}/{name}"

            repo_details = get_repo_details(full_name)

            repo_info = {
                'name': name,
                'owner': owner,
                'url': item.get('html_url'),
                'description': item.get('description'),
                'stars': item.get('stargazers_count'),
                'forks': item.get('forks_count'),
                'created_at': item.get('created_at'),
                'pushed_at': item.get('pushed_at'),
                'primary_language': item.get('language'),
                'all_languages_bytes': repo_details['languages'],
                'topics': ", ".join(repo_details['topics']),
                'contributors_count': repo_details['contributors_count']
            }

            all_final_data.append(repo_info)
            time.sleep(0.5)

    df = pd.DataFrame(all_final_data)
    output_filename = "github_ai_repos_2023_quarters.csv"#سموه باسم السنه حقتكم
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"\n✅ Completed! Saved {len(df)} repositories to {output_filename}")

* 2024

In [ ]:
import requests
import pandas as pd
import time
import random
import re

# ==== CONFIG ====
GITHUB_TOKEN = 'Token'  # حطو حقكم
HEADERS = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
    "User-Agent": "Data-Science-Project-KSUM"
}

QUERY = 'AI OR artificial-intelligence OR machine-learning OR generative-ai'
PER_PAGE = 100
REPOS_PER_QUARTER = 250  # target per quarter
QUARTERS = [
    ('2024-01-01', '2024-03-31'),
    ('2024-04-01', '2024-06-30'),
    ('2024-07-01', '2024-09-30'),
    ('2024-10-01', '2024-12-31')
]#غيرو السنه للسنه حقتكم

# ==== FUNCTIONS ====

def get_repo_details(full_name):
    """Fetch detailed repo info: languages, topics, license, contributors"""
    repo_details = {'languages': {}, 'topics': [], 'license': None, 'contributors_count': 0}

    # Languages
    languages_url = f"https://api.github.com/repos/{full_name}/languages"
    languages_res = requests.get(languages_url, headers=HEADERS)
    if languages_res.status_code == 200:
        repo_details['languages'] = languages_res.json()

    # Topics
    topics_url = f"https://api.github.com/repos/{full_name}/topics"
    topics_headers = HEADERS.copy()
    topics_headers['Accept'] = 'application/vnd.github.mercy-preview+json'
    topics_res = requests.get(topics_url, headers=topics_headers)
    if topics_res.status_code == 200:
        repo_details['topics'] = topics_res.json().get('names', [])

    # License
    repo_url = f"https://api.github.com/repos/{full_name}"
    repo_res = requests.get(repo_url, headers=HEADERS)
    if repo_res.status_code == 200:
        repo_data = repo_res.json()
        if 'license' in repo_data and repo_data['license']:
            repo_details['license'] = repo_data['license'].get('spdx_id')

    # Contributors count
    contributors_url = f"https://api.github.com/repos/{full_name}/contributors?per_page=1"
    contributors_res = requests.get(contributors_url, headers=HEADERS)
    if contributors_res.status_code == 200:
        if 'Link' in contributors_res.headers and 'last' in contributors_res.headers['Link']:
            last_page_url = re.search(r'<(.+)>; rel="last"', contributors_res.headers['Link']).group(1)
            last_page_number = int(re.search(r'page=(\d+)', last_page_url).group(1))
            repo_details['contributors_count'] = last_page_number
        else:
            repo_details['contributors_count'] = len(contributors_res.json())

    return repo_details

def fetch_top_repositories(query, start_date, end_date, per_page=30, max_repos=250):
    """Fetch repositories sorted by stars in a given date range"""
    repos = []
    page = 1
    while len(repos) < max_repos:
        search_query = f'{query} created:{start_date}..{end_date}'
        url = "https://api.github.com/search/repositories"
        params = {
            'q': search_query,
            'sort': 'stars',
            'order': 'desc',
            'per_page': per_page,
            'page': page
        }
        response = requests.get(url, headers=HEADERS, params=params)
        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            break

        items = response.json().get('items', [])
        if not items:
            break

        repos.extend(items)
        page += 1
        time.sleep(1)

    return repos[:max_repos]

# ==== MAIN EXECUTION ====
if __name__ == '__main__':
    all_final_data = []

    for start_date, end_date in QUARTERS:
        print(f"🔍 Fetching top {REPOS_PER_QUARTER} repos from {start_date} to {end_date}...")
        quarter_repos = fetch_top_repositories(QUERY, start_date, end_date, PER_PAGE, REPOS_PER_QUARTER)

        print(f"📦 Collecting detailed info for {len(quarter_repos)} repos...")
        for item in quarter_repos:
            owner = item.get('owner', {}).get('login')
            name = item.get('name')
            full_name = f"{owner}/{name}"

            repo_details = get_repo_details(full_name)

            repo_info = {
                'name': name,
                'owner': owner,
                'url': item.get('html_url'),
                'description': item.get('description'),
                'stars': item.get('stargazers_count'),
                'forks': item.get('forks_count'),
                'created_at': item.get('created_at'),
                'pushed_at': item.get('pushed_at'),
                'primary_language': item.get('language'),
                'all_languages_bytes': repo_details['languages'],
                'topics': ", ".join(repo_details['topics']),
                'contributors_count': repo_details['contributors_count']
            }

            all_final_data.append(repo_info)
            time.sleep(0.5)

    df = pd.DataFrame(all_final_data)
    output_filename = "github_ai_repos_2024_quarters.csv"#سموه باسم السنه حقتكم
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"\n✅ Completed! Saved {len(df)} repositories to {output_filename}")

* 2025

In [ ]:
import requests
import pandas as pd
import time
import random
import re

# ==== CONFIG ====
GITHUB_TOKEN = 'Token'  # حطو حقكم
HEADERS = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
    "User-Agent": "Data-Science-Project-KSUM"
}

QUERY = 'AI OR artificial-intelligence OR machine-learning OR generative-ai'
PER_PAGE = 100
REPOS_PER_QUARTER = 250  # target per quarter
QUARTERS = [
    ('2025-01-01', '2025-03-31'),
    ('2025-04-01', '2025-06-30'),
    ('2025-07-01', '2025-09-23'),
]#غيرو السنه للسنه حقتكم

# ==== FUNCTIONS ====

def get_repo_details(full_name):
    """Fetch detailed repo info: languages, topics, license, contributors"""
    repo_details = {'languages': {}, 'topics': [], 'license': None, 'contributors_count': 0}

    # Languages
    languages_url = f"https://api.github.com/repos/{full_name}/languages"
    languages_res = requests.get(languages_url, headers=HEADERS)
    if languages_res.status_code == 200:
        repo_details['languages'] = languages_res.json()

    # Topics
    topics_url = f"https://api.github.com/repos/{full_name}/topics"
    topics_headers = HEADERS.copy()
    topics_headers['Accept'] = 'application/vnd.github.mercy-preview+json'
    topics_res = requests.get(topics_url, headers=topics_headers)
    if topics_res.status_code == 200:
        repo_details['topics'] = topics_res.json().get('names', [])

    # License
    repo_url = f"https://api.github.com/repos/{full_name}"
    repo_res = requests.get(repo_url, headers=HEADERS)
    if repo_res.status_code == 200:
        repo_data = repo_res.json()
        if 'license' in repo_data and repo_data['license']:
            repo_details['license'] = repo_data['license'].get('spdx_id')

    # Contributors count
    contributors_url = f"https://api.github.com/repos/{full_name}/contributors?per_page=1"
    contributors_res = requests.get(contributors_url, headers=HEADERS)
    if contributors_res.status_code == 200:
        if 'Link' in contributors_res.headers and 'last' in contributors_res.headers['Link']:
            last_page_url = re.search(r'<(.+)>; rel="last"', contributors_res.headers['Link']).group(1)
            last_page_number = int(re.search(r'page=(\d+)', last_page_url).group(1))
            repo_details['contributors_count'] = last_page_number
        else:
            repo_details['contributors_count'] = len(contributors_res.json())

    return repo_details

def fetch_top_repositories(query, start_date, end_date, per_page=30, max_repos=250):
    """Fetch repositories sorted by stars in a given date range"""
    repos = []
    page = 1
    while len(repos) < max_repos:
        search_query = f'{query} created:{start_date}..{end_date}'
        url = "https://api.github.com/search/repositories"
        params = {
            'q': search_query,
            'sort': 'stars',
            'order': 'desc',
            'per_page': per_page,
            'page': page
        }
        response = requests.get(url, headers=HEADERS, params=params)
        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            break

        items = response.json().get('items', [])
        if not items:
            break

        repos.extend(items)
        page += 1
        time.sleep(1)

    return repos[:max_repos]

# ==== MAIN EXECUTION ====
if __name__ == '__main__':
    all_final_data = []

    for start_date, end_date in QUARTERS:
        print(f"🔍 Fetching top {REPOS_PER_QUARTER} repos from {start_date} to {end_date}...")
        quarter_repos = fetch_top_repositories(QUERY, start_date, end_date, PER_PAGE, REPOS_PER_QUARTER)

        print(f"📦 Collecting detailed info for {len(quarter_repos)} repos...")
        for item in quarter_repos:
            owner = item.get('owner', {}).get('login')
            name = item.get('name')
            full_name = f"{owner}/{name}"

            repo_details = get_repo_details(full_name)

            repo_info = {
                'name': name,
                'owner': owner,
                'url': item.get('html_url'),
                'description': item.get('description'),
                'stars': item.get('stargazers_count'),
                'forks': item.get('forks_count'),
                'created_at': item.get('created_at'),
                'pushed_at': item.get('pushed_at'),
                'primary_language': item.get('language'),
                'all_languages_bytes': repo_details['languages'],
                'topics': ", ".join(repo_details['topics']),
                'contributors_count': repo_details['contributors_count']
            }

            all_final_data.append(repo_info)
            time.sleep(0.5)

    df = pd.DataFrame(all_final_data)
    output_filename = "github_ai_repos_2025_quarters.csv"#سموه باسم السنه حقتكم
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"\n✅ Completed! Saved {len(df)} repositories to {output_filename}")